In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import csv
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def get_data(filename):
    with open(filename) as training_file:
        reader = csv.reader(training_file, delimiter=',')    
        imgs = []
        labels = []

        next(reader, None)
        
        for row in reader:
            label = row[0]
            data = row[1:]
            img = np.array(data).reshape((28, 28))

            imgs.append(img)
            labels.append(label)

        images = np.array(imgs).astype(float)
        labels = np.array(labels).astype(float)
    return images, labels


training_images, training_labels = get_data('/kaggle/input/sign-language-mnist/sign_mnist_train.csv')
testing_images, testing_labels = get_data('/kaggle/input/sign-language-mnist/sign_mnist_test.csv')

print(training_images.shape)
print(training_labels.shape)
print(testing_images.shape)
print(testing_labels.shape)

In [ ]:
training_images = np.expand_dims(training_images, axis=3)
testing_images = np.expand_dims(testing_images, axis=3)

# ImageDataGenerator to do Image Augmentation
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
    )

validation_datagen = ImageDataGenerator(rescale=1. / 255)
    
print(training_images.shape)
print(testing_images.shape)

In [ ]:
# Define the CNN model
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(26, activation='softmax')
])

# Compile Model. As this is multi class classification, keep loss as categorical
model.compile(optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])
    
train_gen = train_datagen.flow(
    training_images,
    training_labels,
    batch_size=64
)

val_gen = validation_datagen.flow(
    testing_images,
    testing_labels,
    batch_size=64
)

# Train the Model
history = model.fit_generator(
    train_gen,
    epochs=25,
    validation_data=val_gen
)

model.evaluate(testing_images, testing_labels, verbose=0)


In [ ]:
# Plot the chart for accuracy and loss on both training and validation
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
    
# Predict the label of the test_images
pred = model.predict(testing_images)
pred = np.argmax(pred,axis=1)

# Accuracy score
acc = accuracy_score(testing_labels,pred)

# Print results
print(acc*100)

In [ ]:
# Map the numbers into letters for plotting
alphabets = 'abcdefghijklmnopqrstuvwxyz'
letter_map = {}

for i,j in enumerate(alphabets):
    letter_map[j] = i
letter_map = {v:k for k,v in letter_map.items()}

testing_letters = [letter_map[x] for x in testing_labels]
predicted_letters = [letter_map[x] for x in pred]

In [ ]:
#plotting test images vs predicted values for each sign
correct = np.nonzero(pred == testing_labels)[0]
plt.figure(figsize=(10, 10))
i = 0
for c in correct[:9]:
    plt.subplot(3,3,i+1)
    plt.axis('off')
    plt.imshow(testing_images[c], cmap = "gray", interpolation='none')
    plt.title("{}".format(predicted_letters[c].upper()))
    i += 1

In [ ]:
model.save('/kaggle/working/model_2.h5')
model.save('./model_2.h5')

In [ ]:
!pip install tensorflowjs

import tensorflowjs as tfjs


In [ ]:
tfjs.converters.save_keras_model(model, 'kaggle/working')

In [ ]:

# Modify as needed.
TFJS_MODEL_PATH = '/kaggle/working'

# Save the model in TF's Layer format.
tfjs.converters.save_keras_model(model, TFJS_MODEL_PATH)

In [ ]:
!pip3 install gTTS pyttsx3 playsound gi

In [ ]:
from gtts import gTTS as tts
from playsound import playsound
from IPython.display import Audio
import time
import os
#import os.system

In [ ]:
speech = tts("onuvafnjodb",lang="en")
speech.save("test.mp3")
Audio("./test.mp3",autoplay = True)

In [ ]:
testing_images.shape

In [ ]:
from random import randint
testing_images[randint(0,7171)].reshape(-1,28,28,1).shape

In [ ]:
alphabets = 'abcdefghijklmnopqrstuvwxyz'
def pause():
    programPause = raw_input("Press the <ENTER> key to continue...")
for i in range(2):
    index = randint(0,7171)
    image = testing_images[index].reshape(-1,28,28,1)
    print(alphabets[int(testing_labels[index])].upper())
    a = model.predict(image)
    out  = alphabets[np.where(a == a.max())[1][0]]
    plt.imshow(image[0])
    plt.title(alphabets[int(testing_labels[index])].upper())
    text = "This is the alphabet " + alphabets[int(testing_labels[index])].upper()
    speech = tts(text,lang="en")
    speech.save("test.mp3")
    Audio("./test.mp3",autoplay = True)
    pause()

In [ ]:
!pip install pygame
import pygame
pygame.mixer.init()

music = pygame.mixer.music.load("./test.mp3")
pygame.mixer.music.play()

while (pygame.mixer_music.get_busy()):
    pygame.time.Clock().tick(10)

In [ ]:
Audio("./test.mp3")